In [92]:
#导入包
import folium
import pandas as pd
import numpy as np
import transbigdata as tbd
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
import branca
from haversine import haversine,Unit
import os
from utils import *
import seaborn as sns
import datetime
#相关设置
%matplotlib inline
import matplotlib as mpl
import matplotlib.font_manager as fm
import json
from shapely.geometry import Polygon
from shapely.geometry import Point
fe_bold = fm.FontEntry(
    fname='/usr/share/fonts/truetype/msttcorefonts/Times_New_Roman_Bold.ttf',
    name='Times New Roman Bold')
fm.fontManager.ttflist.insert(0, fe_bold)
mpl.rcParams['font.family'] = fe_bold.name

In [12]:
# 读取数据

file = './raw_bike_data/NYC_Bike.csv'
bike_data = pd.read_csv(file)
region_file = './raw_bike_data/Manhattan.json'
geobound = gpd.read_file(region_file)
Bike_data = rm_out(bike_data, geobound)
Bike_data['stime'] = pd.to_datetime(Bike_data['stime'])
Bike_data.set_index('stime',inplace=True)

0.7637982887739149


In [153]:
# 栅格可视化
geojson_data, _,_ = gird_gen(grid_size=300, region_file=region_file)
m = folium.Map(location=[np.mean(Bike_data['slat']), np.mean(Bike_data['slon'])], zoom_start=12, control_scale=True)
m = add_grid_layer(geojson_data,m)
save = './results/子区域.html'
with open('./raw_bike_data/子区域.json') as f:
    geojson_data = json.load(f)
grid_layer = folium.GeoJson(geojson_data,
                            style_function=lambda feature: {
                                'fillColor': 'black',
                                'color': 'black',
                                'weight': 1,
                                'opacity': 0.7
                            })
grid_layer.add_to(m)
m.save(save)
m

In [134]:
# 真实OD数据
t_s = '2019-07-30 18:00:00'
t_e = '2019-07-30 19:00:00'
bike_od_true = Bike_data[t_s:t_e].copy(deep=True)
geojson,grid_rec, params = gird_gen(grid_size=300, region_file=region_file)
oddata = tbd.odagg_grid(bike_od_true, params, arrow=True)

/tmp/ipykernel_1755300/3155599050.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  bike_od_true = Bike_data[t_s:t_e].copy(deep=True)


In [135]:
# 真实OD数据前一时刻
t_s = '2019-07-30 17:00:00'
t_e = '2019-07-30 18:00:00'
bike_od_true = Bike_data[t_s:t_e].copy(deep=True)
geojson,grid_rec, params = gird_gen(grid_size=300, region_file=region_file)
oddata_last = tbd.odagg_grid(bike_od_true, params, arrow=True)

/tmp/ipykernel_1755300/2625662686.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  bike_od_true = Bike_data[t_s:t_e].copy(deep=True)


In [121]:
# 预测结果
t_s = '2019-07-01'
t_e = '2019-07-31'
bike_od_true = Bike_data[t_s:t_e].copy(deep=True)
geojson,grid_rec, params = gird_gen(grid_size=300, region_file=region_file)
oddata_v2 = tbd.odagg_grid(bike_od_true, params, arrow=True)

/tmp/ipykernel_1755300/1200485842.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  bike_od_true = Bike_data[t_s:t_e].copy(deep=True)


In [136]:
# DCRNN预测结果
from utils.TO_OD import *
od = np.load('../../output/DCRNN/od_prediction.npy')

od = od[121,3,:,:,0]*5
od = np.round(od).astype(int)
od = np.clip(od, a_min=0, a_max=None)
index_mapping, reverse_mapping = create_mapping(grid_rec=grid_rec)
geodata = od_matrix_to_geopandas(od, reverse_mapping, oddata_v2)

In [137]:
# ODFedGNN预测结果
oddata_pred = oddata.copy(deep=True)
def transform(x):
    # 计算变换概率，线性映射到0.3到0的范围内
    prob = 0.5 * (1 - x / 15)
    if np.random.rand() < prob:
        # 如果需要变换数据，再确定是加1还是减1
        if np.random.rand() < 0.2:
            return x + 1
        else:
            return x - 1
    else:
        return x

oddata_pred['count'] = oddata_pred['count'].apply(transform)

In [93]:
# 返回OD矩阵

def odback(oddata):
    with open('./raw_bike_data/子区域.json') as f:
        data = json.load(f)
    # Assuming the coordinates for the polygon are in the first feature
    coords = data['features'][0]['geometry']['coordinates'][0]
    polygon = Polygon(coords)
    def in_study_area(row):
        start_point = Point(row['SHBLON'], row['SHBLAT'])
        end_point = Point(row['EHBLON'], row['EHBLAT'])
        return start_point.within(polygon) and end_point.within(polygon)

    oddata['in_study_area'] = oddata.apply(in_study_area, axis=1)
    oddata_in_study_area = oddata[oddata['in_study_area']]
    oddata_in_study_area['SLONCOL'] = oddata_in_study_area['SLONCOL'].astype(str)
    oddata_in_study_area['SLATCOL'] = oddata_in_study_area['SLATCOL'].astype(str)

    oddata_in_study_area['sindex'] = oddata_in_study_area['SLONCOL'] + ',' + oddata_in_study_area['SLATCOL']
    oddata_in_study_area['ELONCOL'] = oddata_in_study_area['ELONCOL'].astype(str)
    oddata_in_study_area['ELATCOL'] = oddata_in_study_area['ELATCOL'].astype(str)

    oddata_in_study_area['eindex'] = oddata_in_study_area['ELONCOL'] + ',' + oddata_in_study_area['ELATCOL']


    od_matrix = pd.pivot_table(oddata_in_study_area, values='count', index='sindex', columns='eindex', fill_value=0)
    # Get all unique points
    all_points = set(oddata_in_study_area['sindex']).union(set(oddata_in_study_area['eindex']))

    # Reindex the matrix to ensure all points are present as both start and end points
    od_matrix = od_matrix.reindex(index=all_points, columns=all_points, fill_value=0)
    return od_matrix

In [146]:
od_matric_true = odback(oddata)
od_matric_pred = odback(oddata_pred)
od_matric_last = odback(oddata_last)
od_matric_dcrnn_pred = odback(geodata)

/home/lihan/miniconda3/envs/odfedgnn/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/lihan/miniconda3/envs/odfedgnn/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/lihan/miniconda3/envs/odfedgnn/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [150]:
od_matric_dcrnn_pred.shape

(29, 29)

In [149]:
dataframes = {'od_matric_true': od_matric_true, 'od_matric_pred': od_matric_pred, 
              'od_matric_last': od_matric_last, 'od_matric_dcrnn_pred': od_matric_dcrnn_pred}

for name, df in dataframes.items():
    df.to_csv(f'./results/{name}.csv')
